In [1]:
# import dependencies
import pandas as pd
import os
import csv
from sqlalchemy import create_engine

In [2]:
import pymysql
pymysql.install_as_MySQLdb()

In [3]:
# import parks.csv file
park_data=pd.read_csv('parks.csv')
park_data.head()

,Park Code,Park Name,State,Acres,Latitude,Longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


In [4]:
# import species.csv file
species_data=pd.read_csv('species.csv')
species_data.head()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status,Unnamed: 13
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN,NaN
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN,NaN
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern,NaN
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered,NaN
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN,NaN


In [5]:
# merge parks and species data
combined_data=pd.merge(park_data,species_data,on='Park Name',how='inner')

In [6]:
# Familiarize with combined table
combined_data.head()

,Park Code,Park Name,State,Acres,Latitude,Longitude,Species ID,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status,Unnamed: 13
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1000,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN,NaN
1,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1001,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN,NaN
2,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1002,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern,NaN
3,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1003,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered,NaN
4,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1004,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN,NaN


In [7]:
# Look at dimensions of table
combined_data.shape

(119248, 19)

In [8]:
# See which columns have missing values
combined_data.count()

Park Code              119248
Park Name              119248
State                  119248
Acres                  119248
Latitude               119248
Longitude              119248
Species ID             119248
Category               119248
Order                  117776
Family                 117736
Scientific Name        119248
Common Names           119248
Record Status          119248
Occurrence              99106
Nativeness              94203
Abundance               76306
Seasonality             20157
Conservation Status      4718
Unnamed: 13                 5
dtype: int64

In [9]:
# remove columns that have null values 
clean_df = combined_data.dropna(axis=1, how='any')
clean_df.head()

,Park Code,Park Name,State,Acres,Latitude,Longitude,Species ID,Category,Scientific Name,Common Names,Record Status
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1000,Mammal,Alces alces,Moose,Approved
1,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1001,Mammal,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved
2,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1002,Mammal,Canis latrans,"Coyote, Eastern Coyote",Approved
3,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1003,Mammal,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved
4,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1004,Mammal,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved


In [10]:
# Remove unnecessary 'Record Status' column
del clean_df['Record Status']

In [11]:
# Look at dimensions of the table
clean_df.shape

(119248, 10)

In [12]:
# Check how many different common names are present in column for various species
clean_df.loc[clean_df['Scientific Name'] == 'Alces alces']


,Park Code,Park Name,State,Acres,Latitude,Longitude,Species ID,Category,Scientific Name,Common Names
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1000,Mammal,Alces alces,Moose
24768,DENA,Denali National Park and Preserve,AK,3372402,63.33,-150.50,DENA-1003,Mammal,Alces alces,Moose
33458,GAAR,Gates Of The Arctic National Park and Preserve,AK,7523898,67.78,-153.30,GAAR-1002,Mammal,Alces alces,Moose
34813,GLAC,Glacier National Park,MT,1013572,48.80,-114.00,GLAC-1004,Mammal,Alces alces,Moose
37366,GLBA,Glacier Bay National Park and Preserve,AK,3224840,58.50,-137.00,GLBA-1001,Mammal,Alces alces,Moose
52176,GRTE,Grand Teton National Park,WY,309995,43.73,-110.80,GRTE-1004,Mammal,Alces alces,"Eurasian Elk, Moose"
63776,ISRO,Isle Royale National Park,MI,571790,48.10,-88.55,ISRO-1000,Mammal,Alces alces,Moose
67467,KATM,Katmai National Park and Preserve,AK,3674530,58.50,-155.00,KATM-1000,Mammal,Alces alces,Moose
68694,KEFJ,Kenai Fjords National Park,AK,669983,59.92,-149.65,KEFJ-1002,Mammal,Alces alces,Moose
69751,KOVA,Kobuk Valley National Park,AK,1750717,67.55,-159.28,KOVA-1002,Mammal,Alces alces,Moose


In [13]:
clean_df.loc[clean_df['Scientific Name'] == 'Canis lupus']

,Park Code,Park Name,State,Acres,Latitude,Longitude,Species ID,Category,Scientific Name,Common Names
3,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1003,Mammal,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf"
4156,BIBE,Big Bend National Park,TX,801163,29.25,-103.25,BIBE-1010,Mammal,Canis lupus,Gray Wolf
13332,CAVE,Carlsbad Caverns National Park,NM,46766,32.17,-104.44,CAVE-1010,Mammal,Canis lupus,Gray Wolf
19071,CRLA,Crater Lake National Park,OR,183224,42.94,-122.10,CRLA-1008,Mammal,Canis lupus,"Gray Wolf, Wolf"
24771,DENA,Denali National Park and Preserve,AK,3372402,63.33,-150.50,DENA-1006,Mammal,Canis lupus,"Gray Wolf, Wolf"
33462,GAAR,Gates Of The Arctic National Park and Preserve,AK,7523898,67.78,-153.30,GAAR-1006,Mammal,Canis lupus,"Gray Wolf, Wolf"
34819,GLAC,Glacier National Park,MT,1013572,48.80,-114.00,GLAC-1010,Mammal,Canis lupus,Gray Wolf
37369,GLBA,Glacier Bay National Park and Preserve,AK,3224840,58.50,-137.00,GLBA-1004,Mammal,Canis lupus,Gray Wolf
39333,GRBA,Great Basin National Park,NV,77180,38.98,-114.30,GRBA-1011,Mammal,Canis lupus,Gray Wolf
45557,GRSM,Great Smoky Mountains National Park,"TN, NC",521490,35.68,-83.53,GRSM-1008,Mammal,Canis lupus,Gray Wolf


In [14]:
# Delete Common Names column as they are variable
del clean_df['Common Names']
clean_df.head

<bound method NDFrame.head of        Park Code             Park Name State   Acres  Latitude  Longitude  \
0           ACAD  Acadia National Park    ME   47390     44.35     -68.21   
1           ACAD  Acadia National Park    ME   47390     44.35     -68.21   
2           ACAD  Acadia National Park    ME   47390     44.35     -68.21   
3           ACAD  Acadia National Park    ME   47390     44.35     -68.21   
4           ACAD  Acadia National Park    ME   47390     44.35     -68.21   
5           ACAD  Acadia National Park    ME   47390     44.35     -68.21   
6           ACAD  Acadia National Park    ME   47390     44.35     -68.21   
7           ACAD  Acadia National Park    ME   47390     44.35     -68.21   
8           ACAD  Acadia National Park    ME   47390     44.35     -68.21   
9           ACAD  Acadia National Park    ME   47390     44.35     -68.21   
10          ACAD  Acadia National Park    ME   47390     44.35     -68.21   
11          ACAD  Acadia National Park    ME  

In [15]:
clean_df.count()

Park Code          119248
Park Name          119248
State              119248
Acres              119248
Latitude           119248
Longitude          119248
Species ID         119248
Category           119248
Scientific Name    119248
dtype: int64

### Connect to local database

In [16]:
rds_connection_string = "root:password@127.0.0.1/parks_db"
engine = create_engine(f'mysql://{rds_connection_string}')

### Check for tables

In [17]:
engine.table_names()

['Park_data', 'Parks_and_Animals', 'Species_data']

### Use pandas to load csv converted DataFrame into database

In [18]:
clean_df.to_sql(name='Parks_and_Animals', con=engine, if_exists='append', index=False)

In [19]:
species_data.to_sql(name='Species_data', con=engine, if_exists='append', index=False) 

In [20]:
park_data.to_sql(name='Park_data', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table

In [21]:
pd.read_sql_query('select * from Parks_and_Animals', con=engine).head()

,Park Code,Park Name,State,Acres,Latitude,Longitude,Species ID,Category,Scientific Name
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1000,Mammal,Alces alces
1,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1001,Mammal,Odocoileus virginianus
2,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1002,Mammal,Canis latrans
3,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1003,Mammal,Canis lupus
4,ACAD,Acadia National Park,ME,47390,44.35,-68.21,ACAD-1004,Mammal,Vulpes vulpes


In [22]:
pd.read_sql_query('select * from Species_data', con=engine).head()

,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status,Unnamed: 13
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,None,None
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,None,None,None
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,None,Species of Concern,None
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,None,None,Endangered,None
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,None,None


In [23]:
pd.read_sql_query('select * from Park_data', con=engine).head()

,Park Code,Park Name,State,Acres,Latitude,Longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08
